<a href="https://colab.research.google.com/github/fumiya-sakabe/fashion_trend/blob/main/Tops_shirt_trend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Fashion_Trend/Tops_shirt_trend.csv')

df.head()

,月,タンクトップ,Tシャツ,カラーシャツ
0,2004-01,1 未満,12,1 未満
1,2004-02,1 未満,10,1 未満
2,2004-03,1 未満,17,1 未満
3,2004-04,1,22,1 未満
4,2004-05,2,25,1 未満


In [4]:
df['月'] = pd.to_datetime(df['月'])

In [5]:
def month_to_season(month):
    if month == 1 or month == 2 or month == 12:
        return 'winter'
    elif month == 3 or month == 4 or month == 5:
        return 'spring'
    elif month == 6 or month == 7 or month == 8:
        return 'summer'
    else:
        return 'autumn'

In [6]:
df['only_month'] = df['月'].dt.month
df['season'] = df['only_month'].apply(month_to_season)

In [7]:
df

,月,タンクトップ,Tシャツ,カラーシャツ,only_month,season
0,2004-01-01,1 未満,12,1 未満,1,winter
1,2004-02-01,1 未満,10,1 未満,2,winter
2,2004-03-01,1 未満,17,1 未満,3,spring
3,2004-04-01,1,22,1 未満,4,spring
4,2004-05-01,2,25,1 未満,5,spring
...,...,...,...,...,...,...
237,2023-10-01,3,43,2,10,autumn
238,2023-11-01,2,32,1,11,autumn
239,2023-12-01,2,24,1,12,winter
240,2024-01-01,2,25,1,1,winter


In [8]:
df['season'] = pd.Categorical(df['season'], categories=['winter', 'spring', 'summer', 'autumn'], ordered=True)

In [9]:
df['Year'] = df['月'].dt.year

In [10]:
df.replace({"タンクトップ": {"1 未満": 0},"Tシャツ": {"1 未満": 0},"カラーシャツ": {"1 未満": 0}}, inplace=True)

In [11]:
df['タンクトップ'] = df['タンクトップ'].astype('int')
df['Tシャツ'] = df['Tシャツ'].astype('int')
df['カラーシャツ'] = df['カラーシャツ'].astype('int')

In [12]:
df_pivot = df.pivot_table(index=['Year', 'season'], values=df.columns[1:3], aggfunc='mean')

In [13]:
df_pivot

Tシャツ    タンクトップ
Year season                     
2004 winter  10.666667  0.333333
     spring  21.333333  1.000000
     summer  27.666667  1.666667
     autumn  15.666667  0.666667
2005 winter  11.666667  1.000000
...                ...       ...
2023 winter  24.666667  2.000000
     spring  59.000000  4.333333
     summer  85.000000  7.666667
     autumn  44.333333  3.000000
2024 winter  26.000000  2.000000

[81 rows x 2 columns]

In [14]:
growth_rate = df_pivot.pct_change(1).dropna()

In [15]:
growth_rate

Tシャツ    タンクトップ
Year season                    
2004 spring  1.000000  2.000000
     summer  0.296875  0.666667
     autumn -0.433735 -0.600000
2005 winter -0.255319  0.500000
     spring  0.800000  0.333333
...               ...       ...
2023 winter -0.393443 -0.250000
     spring  1.391892  1.166667
     summer  0.440678  0.769231
     autumn -0.478431 -0.608696
2024 winter -0.413534 -0.333333

[80 rows x 2 columns]

In [16]:
trending_items = growth_rate.idxmax(axis=1).reset_index()
trending_items.columns = ['Year', 'season', 'Trending Item']

In [17]:
trending_items

,Year,season,Trending Item
0,2004,spring,タンクトップ
1,2004,summer,タンクトップ
2,2004,autumn,Tシャツ
3,2005,winter,タンクトップ
4,2005,spring,Tシャツ
...,...,...,...
75,2023,winter,タンクトップ
76,2023,spring,Tシャツ
77,2023,summer,タンクトップ
78,2023,autumn,Tシャツ


In [18]:
trending_items.to_csv('/content/drive/MyDrive/Fashion_Trend/Tops_shirt_trend_calu.csv', encoding='shift-jis', index = False)